In [84]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

import os
import allel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score

In [18]:
home = os.path.expanduser('~')
directory = os.path.join('Imp_Research','Dataset')

populations = ['BFcol','BFgam','AOcol','CIcol','CMgam','FRgam',
              'GAgam','GHcol','GHgam','GM','GNcol','GNgam','GQgam',
              'GW','KE','UGgam']
R_SEED = 22

### Functions to Load and Pre-Process the Datasets

In [16]:
'''
Requirements : NumPy and Scikit-Allel
'''

class FilterSNP():
    def __init__(self,haplotype, POS):
        self.haplotype = haplotype
        self.POS = POS
        self.H = haplotype
        self.P = POS
        self._removed_maf = None
        self._retained_maf = None
        self._retained_ld = None
        self._removed_ld = None
        
    def all_filters(self, LD_window_size, LD_overlap_step, MBP_start = 1, MBP_end = 37, MAF_threshold = 5, LD_threshold=.1,LD_iter=1):
        print("1.) Selecting Mega Base Pairs")
        self.H,self.P = self.get_haplo_MBP(self.H,self.P,start = MBP_start, end = MBP_end)
        print("MBP selected. Retained Matrix = ", self.H.shape)
        print("2.) Filtering Rare Allels")
        self.H,self.P = self.filter_MAF(self.H,self.P,threshold = MAF_threshold)
        print("3.) Performing LD Pruning")
        self.H = self.LD_pruning(self.H, LD_window_size, LD_overlap_step, threshold = LD_threshold, n_iter = LD_iter)
        print("Retained Matrix = ", self.H.shape)
        
        return self.H, self.P

    def filter_MAF(self,haplo,POS,threshold = 5):
        if threshold >= 50 : 
            print("MAF threshold cannot be more than 49%")
            return
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = self.get_MAF(haplo)
        #indexes = np.where(maf >= threshold*0.01)[0]
        minor = samples*threshold/100
        major = samples*(100-threshold)/100
        # Selects indexes where allels are >threshold or all 0 and all 1.
        indexes = np.where((sums>=minor)& (sums<=major))[0]
        print("Number of SNPs removed = ",len(haplo)-len(indexes))
        print("Retaining = ",len(indexes))
        self._removed_maf = len(haplo)-len(indexes)
        self._retained_maf = len(indexes)
        return np.take(haplo,indexes,0), np.take(POS,indexes,0)

    # Returns : Array of MAF
    def get_MAF(self,haplo):
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = []
        for s in sums:
            if s != samples or s != 0:
                frequency = s/samples
                if frequency > 0.5:
                    maf.append(1-frequency)
                else: 
                    maf.append(frequency)
        return np.array(maf)

    def get_MBP(self,POS,start = 1,end = 37):
        return np.where(POS[np.where(POS>=1e6)]<=37e6)[0]

    def get_haplo_MBP(self,haplotype,POS,start = 1,end = 37):
        index = self.get_MBP(POS,start,end)
        return np.take(haplotype,index,0),np.take(POS,index,0)
    
    def LD_pruning(self,gn, size, step, threshold = .1, n_iter=1):
        removed = 0
        for i in range(n_iter):
            loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
            n = np.count_nonzero(loc_unlinked)
            n_remove = gn.shape[0] - n
            removed += n_remove
            print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
            gn = gn.compress(loc_unlinked, axis=0)
        self._retained_ld = gn.shape[0]
        self._removed_ld = removed
        return gn

In [12]:
'''
Disclaimer : The class is for personal use. It is not aimed for portability or reusability.

Class to load and filter the data from disk.
Object Parameters : 
data_path  -> Path to the data directory

load_pop() : Function to load the data
params : 
Populations -> list or array of population names/filenames
filtered -> boolean, whethere to filter the data or not
combine -> Boolean, to combine the populations (Implement it !!!)

returns : A dictionary of haplotype matrix and Position array
'''

class LoadFilteredPops():
    def __init__(self,data_path = None):
        self.data_path = data_path
        
    def load_pop(self,populations,naming='custom',chromo_arms = ['3R'],filtered = True):
        if self.data_path is None:
            home = os.path.expanduser('~')
            directory = os.path.join('Imp_Research','Dataset')
        Haplo_pop = {}
        POS_pop = {}
        for population in populations:
            for arm in chromo_arms:
                pop_name = population+'.'+arm
                if naming == 'custom':
                    filename = f'Haplotype.POS.{pop_name}.hd5'
                else:
                    filename = population
                if self.data_path is None:
                    data_path = os.path.join(home, directory,"HDF_Dataset", filename)
                else:
                    try:
                        data_path = os.path.join(self.data_path,filename)
                    except:
                        print("Cannot resolve Directory path")
                        exit()
                print(f'------{pop_name}------\n')
                H = pd.read_hdf(data_path,key='Haplotype').astype('int8').to_numpy().astype('int8')
                P = pd.read_hdf(data_path,key='POS').to_numpy()
                
                if filtered:
                    datafilter = FilterSNP(H,P)
                    Haplo_pop[pop_name],POS_pop[pop_name] = datafilter.all_filters(LD_window_size = 500,LD_overlap_step = 100,LD_iter = 3)
                else: 
                    Haplo_pop[pop_name],POS_pop[pop_name] = H,P
                del H,P     
        print("Populations loaded !!!")
        return Haplo_pop,POS_pop

    
'''
Function to combine the populations
Params
Haplo_all : Dictionary containing Haplotype matrix for populations. Key-> pop name; Value -> matrix.
            Matrix dimensions must be POS x haplotypes.
POS_all : Position array of SNPs
filtered : Boolean, to filter after combining or not.
get_labels : Boolean, to generate labels

returns
Haplo_all : ndarray of n x m dimensions. n = length of POS array; m = sum of haplotypes from all populations.
POS : SNP Position array
labels : list of labels if Labels = []
'''
def combine_pops(H_all,P_all,filtered = True,get_labels = True):
    keys = list(H_all.keys())
    H = np.array(H_all[keys[0]])
    if len(H_all) > 1:
        for i in range(1,len(H_all)):
            H = np.append(H,H_all[keys[i]],axis=1)
    print('Combined Shape => ',H.shape)
    if get_labels:
        label = []
        for each in keys:
            label.extend([each*len(H_all[each][0]))
    if filtered == False:
        if get_labels:
            return H,P_all,label
        else:
            return H,p_all
    else:
        datafilter = FilterSNP(H,P_all)
        H_filtered,POS_filtered = datafilter.all_filters(LD_window_size = 500,LD_overlap_step = 100,LD_iter = 3)
        print('Filtered Shape => ',H_filtered.shape)
        if get_labels:
            return H_filtered,POS_filtered,label
        else:
            return H_filtered,POS_filtered
        

### Loading Dataset

In [9]:
loader = LoadFilteredPops()
Haplo_all, POS_all = loader.load_pop(populations,filtered = False)

------BFcol.3R------

------BFgam.3R------

------AOcol.3R------

------CIcol.3R------

------CMgam.3R------

------FRgam.3R------

------GAgam.3R------

------GHcol.3R------

------GHgam.3R------

------GM.3R------

------GNcol.3R------

------GNgam.3R------

------GQgam.3R------

------GW.3R------

------KE.3R------

------UGgam.3R------

Populations loaded !!!


In [17]:
# Combining and Filtering the SNPs
H_allF, POS_allF, pop_labels = combine_pops(Haplo_all,POS_all['BFcol.3R'],filtered = True)

# To retain the positions simply pass along a list of index values.
# Remove the same indexes as POS hence we will know which indexes to use while infering from models.

Combined Shape =>  (4836295, 2284)
1.) Selecting Mega Base Pairs
MBP selected. Retained Matrix =  (3651720, 2284)
2.) Filtering Rare Allels
Number of SNPs removed =  3451543
Retaining =  200177
3.) Performing LD Pruning
iteration 1 retaining 117993 removing 82184 variants
iteration 2 retaining 117494 removing 499 variants
iteration 3 retaining 117461 removing 33 variants
Retained Matrix =  (117461, 2284)
Filtered Shape =>  (117461, 2284)


In [45]:
# Generating Labels

labels_all = pd.DataFrame(pop_labels)
for i in range(len(populations)):
    labels_all[0].replace(populations[i],i,inplace=True)


# Name coded labels
labels_allname = pd.DataFrame(pop_labels)
labels_allname = labels_allname.to_numpy()
# Number coded class labels
labels_all = labels_all[0].to_numpy()

#### Removing GQgam, GHgam and GNcol due to small sample size

In [53]:
H_13 = H_allF.copy()

remove_pop = np.where((labels_allname == ['GQgam','GHgam','GNcol']))
print("Smaples to be removed",remove_pop[0].shape)

H_13 = np.delete(H_13,remove_pop,1)
print("Samples retained ",H_13.shape)

# Generating new labels

labels_13name = np.delete(labels_allname,remove_pop)
labels_13 = np.delete(labels_all,remove_pop)

Smaples to be removed (50,)
Samples retained  (117461, 2231)


### Generating Train and Test sets

In [87]:
X = H_13.T.copy()
Y = labels_13.copy()

# Patterson Scaled
X_s = allel.PattersonScaler().fit_transform(X)

In [88]:
'''
Train-Test Split
'''

# Stratified Splitting : Representation of all the populations is there in test set.
# Data split : Train = 80%, Test = 20%

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,stratify = Y, random_state = R_SEED)

# x_train and y_train training matrix and training labels
# x_test and y_test testing matrix and test labels
# labels : Array of number coded labels
print("Train set: ",x_train.shape)
print("Test set: ",x_test.shape)

Train set:  (1784, 117461)
Test set:  (447, 117461)


### Linear Discriminant Analysis

In [77]:
lda = LinearDiscriminantAnalysis(solver='svd',n_components = 10)
lda.fit(x_train,y_train)

# lda_s = LinearDiscriminantAnalysis(solver='svd',n_components = 10)
# lda_s.fit(x_train,y_train)

LinearDiscriminantAnalysis(n_components=10)

In [70]:
# we will calculate weighted ROC-AUC in One vs Rest setting
y_pred = lda.predict(x_test)
print("LDA Classification Train Accuracy : ",lda.score(x_train,y_train))
print("LDA Classification Test Accuracy : ",lda.score(x_test,y_test))
print("LDA Classification Test ROC-AUC : ",roc_auc_score(y_test,lda.predict_proba(x_test),multi_class="ovr", average="weighted"))

LDA Classification Train Accuracy :  0.6507847533632287
LDA Classification Test Accuracy :  0.7718120805369127
LDA Classification Test ROC-AUC :  0.9705182962323551


In [78]:
# Patterson Scaled Data
y_pred = lda_s.predict(x_test)
print("LDA Classification Train Accuracy : ",lda_s.score(x_train,y_train))
print("LDA Classification Test Accuracy : ",lda_s.score(x_test,y_test))
print("LDA Classification Test ROC-AUC : ",roc_auc_score(y_test,lda_s.predict_proba(x_test),multi_class="ovr", average="weighted"))

LDA Classification Train Accuracy :  0.640695067264574
LDA Classification Test Accuracy :  0.7718120805369127
LDA Classification Test ROC-AUC :  0.9725032493323238


In [85]:
def cross_validation(clf,X,Y,folds = 5):
    cv_rf = cross_validate(clf,X,Y,scoring=('accuracy','recall','precision'),cv = folds,n_jobs=-1)
    print("Average CV Accuracy Test \t%0.2f"%(cv_rf['test_accuracy'].mean()*100))
#     print("Average CV ROC-AUC Score \t%0.2f"%(cv_rf['test_roc_auc'].mean()*100))
    print("Average CV Recall Score \t%0.2f"%(cv_rf['test_recall'].mean()*100))
    print("Average CV Precision Score \t%0.2f"%(cv_rf['test_precision'].mean()*100))
    print("-----------------------------")

In [86]:
# LDA cross validation 10 Fold
cross_validation(lda,X,Y,folds = 10)

Average CV Accuracy Test 	nan
Average CV Recall Score 	nan
Average CV Precision Score 	nan
-----------------------------


### Logistic Regression

In [93]:
lr = LogisticRegression(solver = 'liblinear',max_iter = 1000,n_jobs = -1,verbose=5)
lr.fit(x_train,y_train)

/home/achal/miniconda3/envs/Imp-Res/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 56.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]

LogisticRegression(max_iter=1000, n_jobs=-1, solver='liblinear', verbose=5)

In [ ]:
y_pred = lda.predict(x_test)
print("LR Classification Train Accuracy : ",lr.score(x_train,y_train))
print("LR Classification Test Accuracy : ",lr.score(x_test,y_test))
print("LR Classification Test ROC-AUC : ",roc_auc_score(y_test,lr.predict_proba(x_test),multi_class="ovr", average="weighted"))

In [ ]:
"""
To Try : 

Data -> UMAP components (3-10) -> Logistic Regression(Any Model)

For inference/predictions the relative components for predictions can be obtained 
from the frozen UMAP model which in turn can be used on LR model.
"""